In [1]:
%matplotlib inline
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import os
home_dir = os.environ['HOME'] + '/'
import pyCloudy as pc

In [2]:
# We define a function that will manage the input files of Cloudy. 
# This allow to easily change some parameters, very usefull to do a grid.
def make_model(dir_, model_name, Teff = 110917.4815262401, L = 35.93721811211741, N=8.14, dens = 3.4771212547196626):
    full_model_name = '{0}_{1:.0f}_{2:.2f}_{3:.2f}_{4:.2f}'.format(model_name, Teff, L, N, dens)
    dist = 6.745 #kpc (from gaia  Bailer-Jones C.A.L.) rgeo
    # these are the commands common to all the models (here only one ...)
    options = ('blackbody ' + str(Teff) + ' linear',
               'luminosity ' + str(L),
               'abundances he=0.112 li=-20 be=-20 b=-20 c=8.81',
                'continue n='+str(N)+ ' o=8.69 f=-20 ne=8.10',
                'continue na=-20 mg=-20 al=-20 si=-20 p=-20',
                'continue s=6.91 cl=-20 ar=6.38 k=-20 ca=-20',
                'continue sc=-20 ti=-20 v=-20 cr=-20 mn=-20',
                'continue fe=-20 co=-20 ni=-20 cu=-20 zn=-20',
               'hden ' + str(dens),
               'radius 16.9030 17.35',
               'no molecules',
               'no level2 lines',
               'no fine opacities',
               'atom h-like levels small',
               'atom he-like levels small',
               'COSMIC RAY BACKGROUND',
               'element limit off -8',
                'print line optical depth', 
               'iterate 2 to convergence max =5',)# 16.9030 17.35 -> set save resolving power 1800
    emis_tab = ['H  1  4861.33A',
            'H  1  6562.81A',
            'Ca B  5875.64A',
            'N  2  6583.45A',
            'O  1  6300.30A',
            'O  2  3726.03A',
            'O  2  3728.81A',
            'O  3  5006.84A',
            'BLND  4363.00A',
            'S  2  6716.44A',
            'S  2  6730.82A',
            'Cl 3  5517.71A',
            'Cl 3  5537.87A',
            'O  1  63.1679m',
            'O  1  145.495m',
            'C  2  157.636m']

    
    # Defining the object that will manage the input file for Cloudy
    c_input = pc.CloudyInput('{0}{1}'.format(dir_, full_model_name))
    # Filling the object with the parameters
    # Defining the ionizing SED: Effective temperature and luminosity.
    # The lumi_unit is one of the Cloudy options, like "luminosity solar", "q(H)", "ionization parameter", etc... 
    #c_input.set_BB(Teff = Teff, lumi_unit = 'q(h)', lumi_value = qH)
    # Defining the density. You may also use set_dlaw(parameters) if you have a density law defined in dense_fabden.cpp.
    #c_input.set_cste_density(dens)
    # Defining the inner radius. A second parameter would be the outer radius (matter-bounded nebula).
    #c_input.set_radius(np.log10(r_min))
    #c_input.set_abund(ab_dict = abund, nograins = True)
    c_input.set_other(options)
    #c_input.set_iterate() # (0) for no iteration, () for one iteration, (N) for N iterations.
    c_input.set_sphere() # () or (True) : sphere, or (False): open geometry.
    c_input.set_emis_tab(emis_tab)
    c_input.set_distance(dist, 'kpc')
    c_input.print_input(to_file = True, verbose = False)

In [3]:
# The directory in which we will have the model
# You may want to change this to a different place so that the current directory
# will not receive all the Cloudy files.
dir_ = '../Models_N/'
os.makedirs(dir_,  exist_ok=True)

In [4]:
# Define verbosity to high level (will print errors, warnings and messages)
pc.log_.level = 3

In [5]:
# Generic name of the models
model_name = 'model'

In [6]:
# tables for the values of the temperature and luminosity
T = [100e3, 110e3, 120e3, 130e3, 140e3, 150e3, 160e3, 170e3, 180e3, 190e3, 200e3]


In [7]:
#L2 #log L = 500 x 3.839×1033 erg/s (in solar units)
    #    L = 700
#L3 #log L = 1 000 x 3.839×1033 erg/s (in solar units)
    #        1 200
    #        1 400
#L5 #log L = 5 000 x 3.839×1033 erg/s (in solar units)
#L4 #log L = 10 000 x 3.839×1033 erg/s (in solar units)
L = []
for i in range(400, 10100, 300):
    ii = np.log10(i*3.839e33)
    L.append(ii)
print(L)

[36.18627810344537, 36.42931615213166, 36.584218112117405, 36.698161464424246, 36.78833809477333, 36.86297171307024, 36.92664079293961, 36.98215812078944, 37.031376143459624, 37.075579805951676, 37.11569702915966, 37.1524198361844, 37.18627810344537, 37.217686567696994, 37.24697594379898, 37.27441419214592, 37.3002214557522, 37.324580801611646, 37.34764610568034, 37.369547947128176, 37.39039808610129, 37.41029291481823, 37.42931615213166, 37.44754097223786, 37.4650317043982, 37.48184520340784, 37.49803196450112, 37.5136370378317, 37.52870078426757, 37.543259504438495, 37.557345965717104, 37.57098984638365, 37.584218112117405]


In [8]:
den = []
for i in range(500, 6000, 500):
    iii = np.log10(i)
    den.append(iii)
print(den)

[2.6989700043360187, 3.0, 3.1760912590556813, 3.3010299956639813, 3.3979400086720375, 3.4771212547196626, 3.5440680443502757, 3.6020599913279625, 3.6532125137753435, 3.6989700043360187, 3.740362689494244]


In [9]:
N = np.arange(7.94, 8.35, 0.03)
N[-1] = 8.34  # Replace the last element with 8.34
print(N)

[7.94 7.97 8.   8.03 8.06 8.09 8.12 8.15 8.18 8.21 8.24 8.27 8.3  8.34]


In [10]:
# defining the models and writing 20 input files
for denn in den:
    for NN in N:
        make_model(dir_, model_name, N=NN, dens=denn)

warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_7.94_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_7.97_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_8.00_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_8.03_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_8.06_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_8.09_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_8.12_2.70.in
warng CloudyInput: "None" parameter not printed
     CloudyInput: Input writen in ../Models_N/model_110917_35.94_8.15_